In [1]:
# syft absolute
import syft as sy
from syft.types.syft_object import SyftObject
from syft import ActionObject

In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)
node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=4d2b70aaa85e4057ae722cdc956e31cf in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=9250cff34b23471ebf4dbe15f3dff756 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

# create datasets

In [4]:
# third party
import numpy as np

mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=private_low,
            shape=private_low.shape,
            mock_is_real=True,
        )
    ],
)

client_low.upload_dataset(dataset_low)

  0%|                                                                                                                 | 0/1 [00:00<?, ?it/s]

Uploading: numpy-data


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.98it/s]


SyftSuccess: Dataset uploaded to 'test_l'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [5]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    id=dataset_low.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.98it/s]


Uploading: numpy-data


SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [6]:
data_low = client_low.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    return data.mean()


compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/a1a51a8fff3a48d3984ed2aac68686e0.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=0fd1752b0ee04629861f2d84ae81b79a in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_1355: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_1355
SyftInfo: Landing the ephmeral node...


```python
Pointer
```
2.0

In [7]:
from time import sleep
sleep(1)

In [8]:
client_low.code.request_code_execution(compute_mean)

syft.service.request.request.Request

In [9]:
request = client_low.requests[0]

In [10]:
code = client_low.code[0]

SyftInfo: Node Landed!


TEO HERE

In [11]:
request.changes

[syft.service.request.request.UserCodeStatusChange]

In [12]:
items_to_sync = [request, code]

- What do we do with properties here?
  - just set them to None

In [13]:
def get_stash_for_item(item, node):
    services = list(node.python_node.service_path_map.values())

    all_stashes = dict()
    for serv in services:
        if (_stash := getattr(serv, "stash", None)) is not None:
            all_stashes[_stash.object_type] = _stash

    stash = all_stashes.get(type(item), None)
    return stash

In [14]:
# job.node_uid

In [15]:
node_high.python_node.id

<UID: 8212e6797fde4c3fba4fc53ab555a886>

In [16]:
def set_obj_ids(x, node_uid, verify_key):
    if hasattr(x, "__dict__") and isinstance(x, SyftObject):
        for prop, val in x.__dict__.items():
            if isinstance(val, (list, tuple)):
                for v in val:
                    set_obj_ids(v, node_uid, verify_key)
            elif isinstance(val, dict):
                for v in val.values():
                    set_obj_ids(v, node_uid, verify_key)
            else:
                set_obj_ids(val, node_uid, verify_key)
        x.syft_node_location=node_uid
        x.syft_client_verify_key=verify_key
        if hasattr(x, "node_uid"):
            x.node_uid=node_uid
                    
                    
    # TODO: hadnle nested collections
            

In [17]:
def create_actionobject(action_object, client):
    action_object = action_object.refresh_object()
    res = client.api.services.action.set(action_object)
    print("Created", res)

In [18]:
def sync_items(items, node, client):
    for item in items_to_sync:
        
        if isinstance(item, ActionObject):
            create_actionobject(item, client)

        else:
            stash = get_stash_for_item(item, node)

            set_obj_ids(item, client.id, client.verify_key)

            res = stash.delete_by_uid(node.python_node.verify_key, item.id)

            res = stash.set(node.python_node.verify_key, item)
            if not res.is_ok():
                raise ValueError("")
            else:
                print("created", item)

In [19]:
sync_items(items_to_sync, node_high, client_high)

created syft.service.request.request.Request
created syft.service.code.user_code.UserCode


# Run code high and sync back result

In [20]:
client_high._fetch_api(client_high.credentials)

In [21]:
data_high = client_high.datasets[0].assets[0]

In [22]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

```python
class Job:
    id: UID = 159bf580d1ae4d43afe4687d0702df3c
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [23]:
# wait for the result
job_high.wait()

```python
Pointer
```
17.0

In [24]:
job_high.result.get()

17.0

In [25]:
# create_actionobject(job_high.result, client_low)

In [26]:
# res = item.wait()
# res.get()
# action_store = node_high.python_node.get_service("actionservice").store
# action_store.set(uid=item.id, credentials=client_high.verify_key, syft_object=item, has_result_read_permission=True)
# stash = get_stash_for_item(item, node_high)
# node_high.action_service

In [27]:
request_high = client_high.requests[0]
code = request_high.code
log = job_high._get_log_objs()

In [28]:
items_to_sync = [code, job_high.result, job_high, log]

In [29]:
# x = job_high.result

In [30]:
# job_high.result.refresh_object()

In [31]:
# job_high.result

In [32]:
# job.node_uid

In [33]:
# log.node_uid

In [34]:
# client_low.notifications[0]

In [35]:
x = client_high.requests[0]

In [36]:
request_high.approve()

Approving request for domain test_h


"Cannot Modify Status as the Domain's data is not included in the request"

In [37]:
# request.node_uid

# request.syft_client_verify_key

# # request

# client_high.id

# client_high.verify_key

In [38]:
# set_obj_ids(request, client_high.id, client_high.verify_key)

# x.changes[0].linked_obj.node_uid

# self.changes.linked_obj



In [39]:
# from syft.client.api import APIRegistry

# for x in APIRegistry.__api_registry__.keys():
#     print(x)

In [40]:
# usercode=list(node_high.python_node.get_service("usercodeservice").stash.partition.data.values())[0]

In [41]:
# usercode.output_policy

In [42]:
sync_items(items_to_sync, node_low, client_low)

created syft.service.code.user_code.UserCode
Created 17.0
created syft.service.job.job_stash.Job
created syft.service.log.log.SyftLog


# Run code low

In [43]:
res_low = client_low.code.compute_mean(data=data_low)

In [44]:
# we need to sync the usercode itself


In [45]:
res_low.get()

7.0

In [46]:
private_high.mean()

17.0

In [47]:
job_low = client_low.code.compute_mean(data=data_low, blocking=False)

SyftWarning: There are existing jobs for this user code, returning the latest one

In [48]:
job_low

```python
class Job:
    id: UID = 159bf580d1ae4d43afe4687d0702df3c
    status: completed
    has_parent: False
    result: 17.0
    logs:

0 
JOB COMPLETED
    
```

In [49]:
job_low.wait().get()

17.0